#  Práctica 6. Redes neuronales Convolucionales para clasificación

Utilizar el dataset MNIST (https://www.kaggle.com/datasets/hojjatk/mnist-dataset), e implementar una red neuronal convolucional que clasifique el dataset mencionado.

Importar modulos

In [ ]:
import tensorflow as tf
import keras
from keras import layers
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

# indica que el número de canales de color se encuentra en la última dimensión de los datos.
keras.backend.set_image_data_format('channels_last')

importar csvs

In [ ]:
for_train = pd.read_csv('./train.csv')
for_test = pd.read_csv('./test.csv')

display("Data entrenamiento", for_train)
display("Data tests", for_test)


Copiar datasets y trabajar con copias

In [ ]:
X_train = for_train.drop('label', axis=1).copy()
X_test = for_test.copy()
Y_train = for_train['label'].copy()

In [ ]:
X_train.describe()

In [ ]:
#para normalizar entre 0 y 255
#siendo 255 el valor con total negro
X_train = X_train / 255.0
X_test = X_test / 255.0

In [ ]:
# Reshape para hacer cuadros de 28 * 28, y hacer mas sencillo el entrenamiento
X_train = X_train.values.reshape(-1, 28, 28, 1)
X_test = X_test.values.reshape(-1, 28, 28, 1)

In [ ]:
#porcentaje para validar el modelo, el resto es enternamiento
porcentaje_para_validacion = 0.3

#y usamos la func para dividir 
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size= porcentaje_para_validacion)

In [ ]:
#Hot encoding =  etiquetas binarias
Y_train = keras.utils.to_categorical(Y_train, num_classes=10)
Y_val = keras.utils.to_categorical(Y_val, num_classes=10)

In [ ]:
def RED_CNN():
    model = keras.Sequential()
    
    ''' 2 capas de convolución con 32 filtros de tamaño 3x3 y un paso de desplazamiento de 1x1. Esto produce 32 mapas de características.
    El padding se establece en 'same', lo que significa que el tamaño de la salida es el mismo que el de la entrada.
    Una capa de normalización por lotes (Batch Normalization) para normalizar las activaciones de la capa anterior.
    Una función de activación ReLU para introducir no linealidad en la red.
    Una capa de Max Pooling para reducir la dimensionalidad de las características.
    Una capa de Dropout para regularizar la red y prevenir el sobreajuste.'''
    model.add(layers.Conv2D(32, (3, 3), (1, 1), padding='valid', input_shape=(28, 28, 1), name='conv2d_1_1'))
    model.add(layers.Conv2D(32, (3, 3), (1, 1), padding='same', name='conv2d_1_2'))
    model.add(layers.BatchNormalization(name='bn_1'))
    model.add(layers.Activation('relu', name='relu_1'))
    model.add(layers.MaxPooling2D((2, 2), (2, 2), padding='valid', name='mp2d_1'))
    model.add(layers.Dropout(0.2, name='drop_1'))
    
    '''Dos capas de convolución con 64 filtros cada una.
    Capas de Batch Normalization y activación ReLU.
    Capas de Max Pooling y Dropout.'''
    model.add(layers.Conv2D(64, (3, 3), (1, 1), padding='valid', name='conv2d_2_1'))
    model.add(layers.Conv2D(64, (3, 3), (1, 1), padding='same', name='conv2d_2_2'))
    model.add(layers.BatchNormalization(name='bn_2'))
    model.add(layers.Activation('relu', name='relu_2'))
    model.add(layers.MaxPooling2D((2, 2), (2, 2), padding='valid', name='mp2d_2'))
    model.add(layers.Dropout(0.2, name='drop_2'))
    
    '''Una capa Flatten para aplanar las características en un solo vector.
    Una capa densa con 100 unidades y activación ReLU.
    Una capa de salida densa con 10 unidades y activación softmax, que se utiliza para clasificar las imágenes en 10 clases diferentes.'''
    model.add(layers.Flatten())
    model.add(layers.Dense(100, activation='relu'))
    model.add(layers.Dense(10, activation='softmax'))
    
    return model

In [ ]:
modelo_CNN = RED_CNN()

In [ ]:
#adam como sustituto de SGD y ajuste automatico a Alfa en training
#la perdida se calcula con ese algo para usar onehot encoding

modelo_CNN.compile(optimizer='adam', loss='CategoricalCrossentropy', metrics=['accuracy'])

In [ ]:
modelo_CNN.summary()

In [ ]:
#Mi compu usa el CPU al 100% y se calienta demasiado jaja, por lo que 3 epochs es sufuciente pa lograr buen accuracy (best = 0.04)

ENTRENAMIENTO = modelo_CNN.fit(X_train, Y_train, validation_data=(X_val, Y_val), batch_size=64, epochs=3, verbose=1)

In [ ]:
#val = validation

plt.subplot(1, 2, 1)
plt.plot(ENTRENAMIENTO.history['accuracy'], label='accuracy')
plt.plot(ENTRENAMIENTO.history['val_accuracy'], label='validation_accuracy')
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend(loc='lower right')

plt.subplot(1, 2, 2)
plt.plot(ENTRENAMIENTO.history['loss'], label='loss')
plt.plot(ENTRENAMIENTO.history['val_loss'], label='validation_loss')
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend(loc='lower right')

plt.tight_layout()
plt.show()

### Fase de prediccion

func para checar prediccion

In [ ]:
def predict(model, X, imgs):
    s = int(np.sqrt(imgs))
    fig, ax = plt.subplots(s, s, sharex=True, sharey=True, figsize=(15, 15))
    ax = ax.flatten()
    preds = model.predict(X[:imgs])
    for i in range(imgs):
        y_pred = np.argmax(preds[i])
        img = X[i].reshape(28, 28)
        ax[i].imshow(img, cmap='Greys', interpolation='nearest')
        ax[i].set_title(f'p: {y_pred}')

In [ ]:
predict(modelo_CNN, X_test, 25)